In [ ]:
import pandas as pd
from mod.config import DATABASE, EVENT_SHEET, MEMBER_SHEET, MENU
from mod import general as gr
from mod import add_member as add
import re

In [ ]:
# 讀取會員表
df_member = gr.GET_DF_FROM_DB(sheet=MEMBER_SHEET)

# 輸入新增會員資料
member_list = add.build_member()

# 將新增資料轉成df
df_temp = pd.DataFrame(member_list)

# 檢查是否有已經加入過會員的資料
df_temp_clean = add.check_and_remove_duplicates(df_member=df_member, df_temp=df_temp)

# 與主表合併
df_member = pd.concat([df_member, df_temp], ignore_index=True)

# 存檔
gr.SAVE_TO_SHEET(df=df_member, sheet=MEMBER_SHEET)


會員資料新增完成！


In [ ]:
class CancelOperation(Exception):
    """使用者手動取消操作"""
    pass


def check_cancel(check: str):
    """輸入「*」可強制中止並取消當前操作"""
    if check == "*":
        raise CancelOperation("使用者取消")


def check_name_in_member(name: str, df_member: pd.DataFrame) -> bool:
    mask1 = (df_member["會員姓名"] == name)
    check_name = df_member[mask1]

    if len(check_name) == 0:
        print("查無此會員姓名，請確認對方是否已加入會員")
        return False

    return True

def check_email_and_name_in_member(email: str, name: str, df_member: pd.DataFrame) -> bool:
    mask1 = (df_member["會員姓名"] == name)
    mask2 = (df_member["Email"] == email)
    check_mail = df_member[mask1 & mask2]

    if len(check_mail) == 0:
        print("查無此會員信箱，請確認是否輸入正確Email或對方已加入會員")
        return False
    elif len(check_mail) > 1:
        print("找到多筆會員資料，請刪除重複資料再繼續")
        return False

    return True

def input_name(df_member: pd.DataFrame):
    name = input("請輸入購買會員姓名：")
    check_cancel(check=name)

    if check_name_in_member(name=name, df_member=df_member):
        return True, name
    else:
        return False, None

def input_email(name: str, df_member: pd.DataFrame):
    email = input("請輸入購買會員Email：")
    check_cancel(check=email)

    if check_email_and_name_in_member(email=email, name=name, df_member=df_member):
        return True, email
    else:
        return False, None

def input_plan():
    while True:
        plan = input("請輸入購買方案：（A.一對一/B.一對二/C.團體）")
        check_cancel(check=plan)

        if plan in ["A", "a", "一對一"]:
            return "A"
        elif plan in ["B", "b", "一對二"]:
            return "B"
        elif plan in ["C", "c", "團體", "團課", "團體課程"]:
            return "C"
        else:
            print("輸入錯誤，請重新輸入")


def input_count(plan: str):
    while True:
        count = input("請輸入購買堂數：（A.四堂/B.八堂/C.十六堂（團體方案不適用））")
        check_cancel(check=count)

        if count in ["A", "a", "四堂", "四", "4", 4]:
            return 4
        elif count in ["B", "b", "八堂", "八", "8", 8]:
            return 8
        elif count in ["C", "c", "十六堂", "十六", "16", 16]:
            if plan == "C":
                print("團體課程單次購買上限為八堂，請重新輸入")
            else:
                return 16
        else:
            print("輸入錯誤，請重新輸入")


def input_price(plan:str, count:int):
    df_menu = gr.GET_DF_FROM_DB(sheet=MENU)
    df_menu["price"] = df_menu["price"].astype(int)

    mask1 = (df_menu["plan"] == plan)
    mask2 = (df_menu["count"] == count)
    price = df_menu[mask1 & mask2]["price"]
    total = price * count

    return price, total

def input_payment():
    payment = input("請輸入付款方式：（A.現金/B.匯款/C.其他）")
    if payment in ["A", "a", "現金", "付現"]:
        return "現金"
    elif payment in ["B", "b", "轉帳", "匯款"]:
        return "匯款"
    elif payment in ["C", "c", "其他"]:
        return "其他"

def input_account_id(payment: str):
    if payment == "現金":
        return "無"
    else:
        while True:
            account_id = input("請輸入付款帳號末五碼：")
            check_cancel(check=account_id)

            ACCOUNT_ID = r"^[0-9]{5}$"
            if re.fullmatch(ACCOUNT_ID, account_id):
                return account_id
            else:
                print("輸入格式錯誤，請重新輸入帳號末五碼")


def purchase_plan():
    check = True
    while check:
        purchase_info = {}

        check, name = input_name(df_member=df_member)
        purchase_info["會員姓名"] = name

        check, email = input_email(name=name, df_member=df_member)
        purchase_info["Email"] = email

        plan = input_plan()
        purchase_info["方案"] = plan

        count = input_count(plan=plan)
        purchase_info["堂數"] = count

        price, total = input_price(plan=plan, count=count)
        purchase_info["單堂金額"] = price
        purchase_info["方案總金額"] = total

        payment = input_payment()
        purchase_info["付款方式"] = payment

        account_id = input_account_id(payment=payment)
        purchase_info["匯款末五碼"] = account_id

        return purchase_info


def keep_order_or_not():
    while True:
        keep_or_not = input("是否繼續新增購買訂單？[Y/n]").lower()
        if keep_or_not in ["n", ""]:
            return False
        elif keep_or_not == "y":
            return True
        else:
            print("輸入錯誤，請重新輸入")


def build_order():
    order_list = []

    while True:
        try:
            purchase_info = purchase_plan()
            order_list.append(purchase_info)

            keep = keep_order_or_not()
            if not keep:
                return order_list

        except CancelOperation as e:
            print(f"{e}")
            return order_list

In [ ]:
# 讀取事件紀錄表
df_event = gr.GET_DF_FROM_DB(sheet=EVENT_SHEET)

# 讀取會員表
df_member = gr.GET_DF_FROM_DB(sheet=MEMBER_SHEET)

# 建立訂單列表
order_list = build_order()

# 將訂單列表轉換成df
df_temp = pd.DataFrame(order_list)






In [ ]:
columns = ['會員姓名', 'Email', '方案', '堂數', '單堂金額', '方案總金額', '付款方式', '匯款末五碼']
df_event

Index(['會員姓名', 'Email', '生日', '方案', '堂數', '單堂金額', '方案總金額', '付款方式', '匯款末五碼'], dtype='object')